<a href="https://colab.research.google.com/github/cosshoon/Anomaly-Detection-/blob/main/Previous_works/CNN%EB%AA%A8%EB%8D%B8_Hyper_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#지난 과제 5에서 진행했던 데이터셋 및 CNN 모델 활용하여, 파라미터 튜닝 등 성능향상 시도
# airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck 의 10개의 카테고리가 각각 6000장씩 있는 데이터셋
import tensorflow as tf
from tensorflow import keras 

import numpy as np
import matplotlib.pyplot as plt

cifar10 = keras.datasets.cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#train set 중에서 10000개를 validation set으로 나눔

x_val, y_val = x_train[40000:,:,:], y_train[40000:]
x_train, y_train = x_train[:40000,:], y_train[:40000,:]

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
# 데이터 표준화
import numpy as np
mean = np.mean(x_train, axis=0)
print("mean.shape =",mean.shape)
std = np.std(x_train)
print("std =",std)

x_train_centered = (x_train-mean)/std
x_val_centered = (x_val-mean)/std
x_test_centered = (x_test-mean)/std

mean.shape = (32, 32, 3)
std = 64.19169405614137


In [ ]:
# one hot encoding
from tensorflow.keras.utils import to_categorical

y_train_onehot = to_categorical(y_train)
y_val_onehot = to_categorical(y_val)
y_test_onehot = to_categorical(y_test)

In [ ]:
from tensorflow.keras import layers, models

In [ ]:
model1 = models.Sequential()

model1.add(layers.Conv2D(32,(5,5), padding='valid', activation='relu', input_shape=(32,32,3)))

model1.add(layers.MaxPool2D(2,2))
model1.add(layers.Conv2D(64,(5,5), padding='valid', activation='relu'))
model1.add(layers.Conv2D(128,(2,2), padding='valid', activation='relu'))
model1.add(layers.MaxPool2D(2,2))
model1.add(layers.Flatten())

# dense layer에 넣기
model1.add(layers.Dense(512, activation='relu'))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(10, activation='softmax'))

model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1

In [ ]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history=model1.fit(x_train_centered, y_train_onehot, batch_size=32, epochs=20,
                  validation_data=(x_val_centered, y_val_onehot))

Epoch 1/20
1250/1250 [==============================] - 13s 10ms/step - loss: 0.2395 - acc: 0.9279 - val_loss: 2.0924 - val_acc: 0.6712
Epoch 2/20
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2259 - acc: 0.9310 - val_loss: 2.0631 - val_acc: 0.6795
Epoch 3/20
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2228 - acc: 0.9293 - val_loss: 2.1070 - val_acc: 0.6686
Epoch 4/20
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2416 - acc: 0.9249 - val_loss: 2.1153 - val_acc: 0.6785
Epoch 5/20
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2303 - acc: 0.9296 - val_loss: 2.0421 - val_acc: 0.6769
Epoch 6/20
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2290 - acc: 0.9315 - val_loss: 2.2031 - val_acc: 0.6692
Epoch 7/20
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2373 - acc: 0.9255 - val_loss: 2.3398 - val_acc: 0.6690
Epoch 8/20
1250/1250 [==============================] 

In [ ]:
#하이퍼파라미터 튜닝을 통해 성능향상 시켜보기

!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.0MB/s 


In [ ]:
import kerastuner as kt

In [ ]:
def model_builder(hp):
  model2 = models.Sequential()
  model2.add(layers.Conv2D(32,(5,5), padding='valid', activation='relu', input_shape=(32,32,3)))

  model2.add(layers.MaxPool2D(2,2))
  model2.add(layers.Conv2D(64,(5,5), padding='valid', activation='relu'))
  model2.add(layers.Conv2D(128,(2,2), padding='valid', activation='relu'))
  model2.add(layers.MaxPool2D(2,2))
  model2.add(layers.Flatten())
  
  # Dense Layer의 Output size를 바꾸기 위한 변수 선언
  hp_units = hp.Int('units', min_value = 512, max_value = 2048, step = 512)

  # Dense layer에 넣기
  model2.add(layers.Dense(units = hp_units, activation='relu'))
  model2.add(layers.Dropout(0.5))
  model2.add(layers.Dense(10, activation='softmax'))

  model2.summary()

  #Adam Optimizer 학습률 선택지
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  #Optimizer에 학습률 변수를 넣기 위해 keras.optimziers.Adam 사용
  model2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss='categorical_crossentropy', 
                metrics = ['accuracy'])
  return model2


In [ ]:
# 인스턴스 만들기
tuner = kt.RandomSearch(model_builder,
                     objective = 'val_accuracy', 
                     max_trials = 20)  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1

In [ ]:
tuner.search(x_train_centered, y_train_onehot, epochs = 20, validation_data = (x_val_centered, y_val_onehot))

Trial 10 Complete [00h 03m 26s]
val_accuracy: 0.10249999910593033

Best val_accuracy So Far: 0.7289999723434448
Total elapsed time: 00h 37m 08s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hps.values

{'learning_rate': 0.0001, 'units': 1536}

In [ ]:
model2 = tuner.hypermodel.build(best_hps)

history2=model2.fit(x_train_centered, y_train_onehot, batch_size=32, epochs=20,
                  validation_data=(x_val_centered, y_val_onehot))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1536)              3

In [ ]:
#테스트 이미지로 모델 1과 모델 2 성능 확인 및 비교

test_loss, test_acc = model1.evaluate(x_test_centered, y_test_onehot, verbose=2)

test_loss, test_acc = model2.evaluate(x_test_centered, y_test_onehot, verbose=2)

313/313 - 1s - loss: 2.5539 - acc: 0.6737
313/313 - 1s - loss: 0.9361 - accuracy: 0.7235


In [ ]:
# model 1에 비해서 하이퍼파라미터 튜닝을 진행한  model 2가 테스트셋의 정확도가 0.6737에서 0.7235로 증가한 것을 확인할 수 있다.